In [362]:
import csv
import pandas as pd
import numpy as np
import requests
from IPython.core.display import HTML
from selenium import webdriver
from bs4 import BeautifulSoup
import timeit
import urllib2
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline


In [173]:
###Scraping Data (Takes a couple minutes)
salary = []
company = []
location = []
jobs = []
jobdescription = []
for i in range(31):
    url = "http://www.careerbuilder.com/jobs-data-science?page_number=1&pay=20"
    index = 59
    char = i
    charplus = i + 1
    charplusstring = str(charplus)
    url2 = url[:index] + charplusstring + url[index + 1:]
    rm = requests.get(url2)
    soupm = BeautifulSoup(rm.content, "lxml")
    
    ###Appends job titles
    for d in soupm.findAll(class_='job-title'):
        jobs.append(d.text)
        
    ###Appends job descriptions
    for d in soupm.findAll(class_='job-description show-for-medium-up'):
        jobdescription.append(d.text)
    
    ###Appends salary, company, location.
    z = 0
    for d in soupm.findAll(class_='job-text'):
        if z % 3 == 0:
            salary.append(d.text)
        elif z % 3 == 1:
            company.append(d.text)
        else:
            location.append(d.text)
        
        z += 1
        

In [9]:
###Returning the lists to check
#salary
#company
#location
#jobs
#jobdescription

In [174]:
###Zipping them all together
info = zip(salary, company, location, jobs, jobdescription)

In [175]:
###Putting in DataFrame
df = pd.DataFrame(info)

In [176]:
###Naming Columns
df.columns = ('salary', 'company', 'location', 'title', 'description')

In [177]:
###Breaking location into State/City
df['state'] = [(x[-3:-1]) for x in df['location']]

In [178]:
###Breaking location into City
df['delete'] = df['location'].str.replace('\n', '-------------------------')

In [179]:
df['delete2'] = [(x[-80:-29]) for x in df['delete']]

In [180]:
df['delete3'] = df['delete2'].str.replace('-', '')

In [181]:
df['city'] = df['delete3']

In [182]:
del df['delete']
del df['delete2']
del df['delete3']
del df['location']

In [183]:
###Removing \n from company
df['company'] = df['company'].str.replace('\n', '')

In [184]:
###Removing \n from title
df['title'] = df['title'].str.replace('\n', '')

In [185]:
###Removing \n from description
df['description'] = df['description'].str.replace('\n', '')

In [186]:
###Creating a column for low end of salary
df['low'] = [(x[18:22]) for x in df['salary']]

In [187]:
###Removing '$'
df['low'] = df['low'].str.replace('$', '')

In [188]:
###Removing 'k'
df['low'] = df['low'].str.replace('k', '')

In [189]:
###Removing '-'
df['low'] = df['low'].str.replace('-', '')

In [190]:
###Removing ','
df['low'] = df['low'].str.replace(',', '')

In [191]:
###Creating a column for high end of salary
df['high'] = [(x[-11:-6]) for x in df['salary']]

In [192]:
###Removing '$'
df['high'] = df['high'].str.replace('$', '')

In [193]:
###Removing 'k'
df['high'] = df['high'].str.replace('k', '')

In [194]:
###Removing '-'
df['high'] = df['high'].str.replace('-', '')

In [195]:
###Removing ','
df['high'] = df['high'].str.replace(',', '')

In [196]:
###Fixing '\nFull'
df['high'] = df['high'].str.replace('\nFull', '')

In [72]:
df.dtypes

salary         object
company        object
title          object
description    object
state          object
city           object
low            object
high           object
dtype: object

In [198]:
###START OF CODE TO FIX NANS AND CONVERT TO INTEGERS
dftest = df.copy()

In [199]:
###Turning emptys into nans and dropping
dftest['high'].replace('', np.nan, inplace=True)
dftest['low'].replace('', np.nan, inplace=True)
dftest.dropna(subset=['high'], inplace=True)
dftest.dropna(subset=['low'], inplace=True)

In [201]:
###Turning remaining emptys into 0
dftest['high']=dftest['high'].replace('', '0')
dftest['low']=dftest['low'].replace('', '0', inplace=True)

In [503]:
###Overriding error to return 0s and ints
testing = df['high']

In [211]:
def replace_nulls(x):
    try:
        return int(x)
    except:
        '0'

In [212]:
testing = testing.apply(replace_nulls)

In [208]:
#testing.dropna(inplace=True)

In [214]:
lowtest = df['low']

In [215]:
lowtest = lowtest.apply(replace_nulls)

In [134]:
# lowtest.dropna(inplace=True)

In [152]:
# lowtest

In [217]:
###putting together for a dataframe
salarycolumns = zip(lowtest, testing)

In [218]:
salarycolumnsframe = pd.DataFrame(salarycolumns)

In [219]:
df2 = pd.concat([df, salarycolumnsframe], axis=1)

In [224]:
#Renaming Columns
df2.columns = ('salary', 'company', 'title', 'description', 'state', 'city', 'low', 'high', 'lowsalary', 'highsalary')

In [226]:
###Dropping Columns
del df2['salary']
del df2['low']
del df2['high']

In [234]:
#Getting Rid of Error Salaries
def drop_outliers(x):
    if x > 250:
        return np.nan
    else:
        return x

In [235]:
fixer10 = df2['highsalary']

In [237]:
fixer10 = fixer10.apply(drop_outliers)

In [239]:
#Getting Rid of Error Salaries in the 'Low' Column
def low_outliers(x):
    if x > 5:
        return x
    else:
        return np.nan

In [240]:
lowerfixer = df2['lowsalary']

In [248]:
lowhighfix = zip(lowerfixer, fixer10)

In [249]:
lowhigh = pd.DataFrame(lowhighfix)

In [251]:
df2 = pd.concat([df2, lowhigh], axis=1)

In [259]:
df2['low_salary'] = df2[0]
df2['high_salary'] = df2[1]
#df2.head()

In [260]:
##Dropping Columns
del df2[0]
del df2[1]
del df2['lowsalary']
del df2['highsalary']

In [266]:
#Average Salary
((df2['low_salary'].mean()) + (df2['high_salary'].mean())) / 2

95.04615780155692

In [295]:
###Replacing Nans with Average
def averages(x):
    try:
        return int(x)
    except:
        return 95.0

In [296]:
tester94 = df2['low_salary']

In [297]:
tester94 = tester94.apply(averages)

In [300]:
low_with_average = tester94

In [302]:
###Return the mean for 0.0 cells
def zerofixer(x):
    try:
        return (x / 0) + x
    except:
        return 95.0

In [304]:
low_with_average = low_with_average.apply(zerofixer)

In [308]:
low_testing = tester94

In [311]:
toaddagain = pd.DataFrame(low_testing)

In [313]:
df3 = pd.concat([df2, toaddagain], axis=1)

In [315]:
df4 = df3.copy()

In [317]:
df5 = df2.copy()

In [319]:
###Applying the original 'averages' function directly to the dataframe
df5['low_salary'] = df5['low_salary'].apply(averages)

In [321]:
df5['high_salary'] = df5['high_salary'].apply(averages)

In [323]:
df6 = df5.copy()

In [350]:
###Easier method. Returning the mean if salary is less than $5,000.
def zeros_to_average(x):
    if x < 5:
        return 95.0
    else:
        return x

In [356]:
df5['low_salary'] = df5['low_salary'].apply(zeros_to_average)
df5['average_salary'] = (df5['low_salary'] + df5['high_salary']) / 2

In [358]:
dfreg = df5.copy()

In [360]:
del dfreg['low_salary']
del dfreg['high_salary']

In [504]:
dfreg

,company,title,description,state,city,average_salary,high_vs_low
0,Sancus Associates,Associate Data Science,"Associate Data Scientist AdTech $125,000-150,0...",DC,District of Columbia,137.5,
1,Analytic Recruiting,"Associate, data Science and market Intelligence","Associate, data Science and market Intelligenc...",NY,New York,100.0,
2,Tech Mahindra,BI Data Engineer,Position Title: BI Data Engineer Location: New...,OH,New Albany,95.0,
3,Challenge Charter School,6th grade Science & Language Arts Teacher - Mi...,Arizona’s 1st Official Core Knowledge® School;...,AZ,Glendale,41.0,
4,CyberCoders,Senior Data Scientist - be the first Data Scie...,This position is open as of 10/18/2016. Senior...,CA,Marina del Rey,175.0,
5,Medix,Research Scientist,Research Scientist This Research Scientist wil...,TX,Dallas,60.0,
6,BIG WEDNESDAY DIGITAL,Junior Data Scientist - global eCommerce - Ban...,Junior Data Scientist - global eCommerce leade...,MD,Baltimore,97.5,
7,Burnett Specialists,Data Scientist,Job Reference #: 41-1014TBD-16CT Data Scientis...,TX,Houston,100.0,
8,BIG WEDNESDAY DIGITAL,Senior/Lead/NLP- Data Scientist - Bangkok role...,Senior/Lead/NLP- Data Scientist - Bangkok base...,WA,Seattle,97.5,
9,Kelly Scientific Resources,"Sr. Manager, Medical Data Methodology",Kelly Clinical and a highly reputable pharmace...,MA,Cambridge,110.0,


In [363]:
dfreg['high_vs_low'] =

In [365]:
##Bucketing Income
def bucket(x):
    if x['average_salary'] > 95:
        return 1
    else:
        return 0

In [366]:
dfreg2 = dfreg.copy()

In [367]:
dfreg2['high_vs_low'] = dfreg2.apply(bucket, axis =1)

In [387]:
##Now have complete dataset
dfreg2['label'] = dfreg2['high_vs_low']

In [389]:
del dfreg2['high_vs_low']

In [392]:
data = dfreg2

<h1> 'Data' is our final dataset </h1>
<br>
<p>
After scraping 30 pages of search results for "Data Science" on careerbuilder.com, cleaning the data, and creating a binary column for 'high' vs 'low' salary we finally have our final dataframe. We determined salaries above the mean of $95,000 would be considered 'high' and anything else would be 'low'.
</p>

In [505]:
data.head()

,company,title,description,state,city,average_salary,label
0,Sancus Associates,Associate Data Science,"Associate Data Scientist AdTech $125,000-150,0...",DC,District of Columbia,137.5,1
1,Analytic Recruiting,"Associate, data Science and market Intelligence","Associate, data Science and market Intelligenc...",NY,New York,100.0,1
2,Tech Mahindra,BI Data Engineer,Position Title: BI Data Engineer Location: New...,OH,New Albany,95.0,0
3,Challenge Charter School,6th grade Science & Language Arts Teacher - Mi...,Arizona’s 1st Official Core Knowledge® School;...,AZ,Glendale,41.0,0
4,CyberCoders,Senior Data Scientist - be the first Data Scie...,This position is open as of 10/18/2016. Senior...,CA,Marina del Rey,175.0,1


In [498]:
###Testing to see which words in 'Job Titles' correspond to higher salaries

from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(
    binary=True,  
    stop_words='english', 
    max_features=50, 


X = v.fit_transform(data.title).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
X.head()

,administrator,analyst,application,applications,architect,associate,bangkok,bi,business,clinical,...,senior,software,specialist,sql,sr,supervisor,support,systems,technical,web
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [499]:
y = data.label

In [500]:
#Function to help examine the coefficients
def examine_coefficients(model, df9):
    df9 = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : df9.columns}
    ).sort_values(by='Coefficient')
    return df9[df9.Coefficient !=0 ]

In [501]:
###Creates DataFrame of Coefficients for Job Title
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty = 'l1', C = 1.0) # Model with high, little regularization

model.fit(X, y)
datatitle = examine_coefficients(model, X)

In [422]:
###Testing to see how well the model performed
from sklearn.cross_validation import cross_val_score

for metric in ['accuracy', 'precision', 'recall', 'roc_auc']:
    scores = cross_val_score(model, X, y, scoring=metric)
    print("mean {}: {}, all: {}".format(metric, scores.mean(), scores))
    
# mean roc_auc: 0.829

mean accuracy: 0.682586149214, all: [ 0.67346939  0.67346939  0.70081967]
mean precision: 0.688739061899, all: [ 0.68817204  0.69662921  0.68141593]
mean recall: 0.590363590135, all: [ 0.55652174  0.53913043  0.6754386 ]
mean roc_auc: 0.74445422363, all: [ 0.7451505   0.72458194  0.76363023]


<h1> Testing the Model </h1>
<br>
<p>
Our model testing words in 'job title' to predict salary had an roc_auc score of 0.74, accuracy of .68, precision of .68, and recall of .59.
</p>
<p> The words in the job title that were the strongest indicators of low salary were 'level', 'coordinator', 'electrical', 'analyst', 'quality', 'supervisor', 'associate', and 'support'.
</p>
<p>
The words in the job title that were the strongest indicators of high salaries were 'architect', 'infrastructure', 'director', 'manager', 'sr', 'senior', 'lead', and 'development'
</p>
<p>
The words in job description that were the strongest indicators of high salaries were 'manager', 'responsible', 'experience', 'senior', 'business',  and 'growing'.
</p>
<p>
The words in job description that were the strongest indicators of low salaries were 'analyst', 'position', 'job', 'support', 'opportunity', 'hire', 'new', 'systems', and 'time'.
</p>

In [502]:
datatitle

,Coefficient,Feature
26,-2.011881,level
10,-1.251830,coordinator
18,-0.736807,electrical
1,-0.703504,analyst
36,-0.382542,quality
45,-0.297347,supervisor
5,-0.194904,associate
46,-0.138953,support
28,-0.087896,marketing
42,-0.087054,specialist


In [496]:
datadescription

,Coefficient,Feature
5,-0.993698,analyst
32,-0.459877,position
23,-0.309967,job
43,-0.226324,support
31,-0.222420,opportunity
21,-0.215108,hire
29,-0.213686,new
44,-0.182652,systems
47,-0.157898,time
48,-0.139719,work


In [435]:
###Model based on Description
v = CountVectorizer(
    binary=True,  # Create binary features
    stop_words='english', # Ignore common words such as 'the', 'and'
    max_features=50, # Only use the top 50 most common words
)


# This builds a matrix with a row per website (or data point) and column per word (using all words in the dataset)
X = v.fit_transform(data.description).todense()
X = pd.DataFrame(X, columns=v.get_feature_names())
X.head()

,00,000,10,18,2016,analyst,applications,based,business,classification,...,software,solutions,summary,support,systems,team,technology,time,work,year
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [436]:
def examine_coefficients2(model, df10):
    df10 = pd.DataFrame(
        { 'Coefficient' : model.coef_[0] , 'Feature' : df10.columns}
    ).sort_values(by='Coefficient')
    return df10[df10.Coefficient !=0 ]

In [439]:
model = LogisticRegression(penalty = 'l1', C = 1.0) # Model with high, little regularization

model.fit(X, y)
datadescription = examine_coefficients(model, X)

In [495]:
#data

In [448]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import patsy

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.grid_search import GridSearchCV

In [479]:
###We also created a model based on City/State/Title/Description
X = patsy.dmatrix('~C(city) + C(state) +C(company) + C(title) +C(description)' , data)
y = data['label'].values

In [480]:
pdf = pd.DataFrame(X, columns=X.design_info.column_names)
pdf['Target'] = y
#pdf

In [481]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=77)

In [482]:
lr = LogisticRegression(solver='liblinear')

In [483]:
lr_model = lr.fit(X_train, y_train)

In [484]:
lr_ypred = lr_model.predict(X_test)

In [485]:
# actual = index; predicted = columns
lr_cm = confusion_matrix(y_test, lr_ypred, labels=lr.classes_)
lr_cm = pd.DataFrame(lr_cm, columns=lr.classes_, index=lr.classes_)
lr_cm

,0,1
0,100,40
1,39,64


In [486]:
print classification_report(y_test, lr_ypred, labels=lr.classes_)

             precision    recall  f1-score   support

          0       0.72      0.71      0.72       140
          1       0.62      0.62      0.62       103

avg / total       0.68      0.67      0.68       243



In [523]:
lr_l1 = LogisticRegression(C=500, penalty='l1', solver='liblinear')
lr_l1_model = lr_l1.fit(X_train, y_train)

In [524]:
lr_l1_model = lr_l1.fit(X_train, y_train)

In [525]:
lr_l1_ypred = lr_l1_model.predict(X_test)

In [526]:
lr_l1_cm = confusion_matrix(y_test, lr_l1_ypred, labels=lr_l1.classes_)
lr_l1_cm = pd.DataFrame(lr_l1_cm, columns=lr_l1.classes_, index=lr_l1.classes_)
lr_l1_cm

,0,1
0,112,28
1,49,54


In [527]:
print classification_report(y_test, lr_l1_ypred, labels=lr_l1.classes_)

             precision    recall  f1-score   support

          0       0.70      0.80      0.74       140
          1       0.66      0.52      0.58       103

avg / total       0.68      0.68      0.68       243



<p>
Our model testing based on city, state, title, description, and company had an f1-score of .68 when we did a regular logistic regression and had an f1-score of .68 when we used an L1 with a C of 500.
</p>